In [1]:
import os
import sys
import copy
import json
import pickle

import numpy as np
import rmgpy.chemkin
import rmgpy.tools.uncertainty

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def unpack_sensitivity(long_desc):
    start_str = 'sensitivities = '
    if start_str not in long_desc:
        return []
    start_index = long_desc.find(start_str) + len(start_str)
    sensitivities_str = long_desc[start_index:].replace("'", '"')
    sensitivities_str = sensitivities_str.replace("nan", '"-9999999"')
    sensitivities_str = sensitivities_str.replace('name', 'training_rxn_name')
    return json.loads(sensitivities_str)

In [3]:
# Load the RMG mechanism

# Load the base model and the covariance matrix
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_model'
# basedir = '/home/moon/autoscience/autoscience/butane/models/rmg_model'

base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')

species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)

covariance_file = '/work/westgroup/harris.se/autoscience/autoscience/uncertainty/butane_covariance.pickle'
# covariance_file = '/home/moon/autoscience/autoscience/uncertainty/butane_covariance.pickle'
with open(covariance_file, 'rb') as handle:
    Sigma_k = pickle.load(handle)


# Load the RMG Database

In [ ]:
# load the model

uncertainty = rmgpy.tools.uncertainty.Uncertainty(output_directory='rmg_uncertainty')
uncertainty.load_model(base_chemkin, dictionary)




# TODO - force the user to provide the input file used to generate the mechanism to ensure databases are really the same
# load the database
# --------------- CAUTION!!! Databases here must match the ones used to generate the mechanism
# note - this cell stalls out on Discovery
thermo_libs = [
#     'BurkeH2O2',
    'primaryThermoLibrary',
#     'FFCM1(-)',
#     'CurranPentane',
#     'Klippenstein_Glarborg2016',
#     'thermo_DFT_CCSDTF12_BAC',
#     'DFT_QCI_thermo',
#     'CBS_QB3_1dHR',
]

kinetic_libs = [
#     'FFCM1(-)',
#     'CurranPentane',
#     'combustion_core/version5',
#     'Klippenstein_Glarborg2016',
#     'BurkeH2O2inArHe',
#     'BurkeH2O2inN2',
]
uncertainty.load_database(
    thermo_libraries=thermo_libs,
#     kinetics_families='default',
    kinetics_families=['Disproportionation'],
    reaction_libraries=kinetic_libs,
    kinetics_depositories=['training'],
)


In [ ]:
# don't just count the nodes to get popularity. Use the sensitivity of each reaction

# start by loading sensitivity of reactions
rxn_sensitivity_file = '/home/moon/autoscience/autoscience/butane/sensitivity_analysis/base_rmg24/reaction_ignition_delay_plain.npy'
base_delay = 0.14857535
ignition_delay = np.load(rxn_sensitivity_file)
sensitivities = np.abs(ignition_delay - base_delay) / base_delay

In [ ]:
# Get the different kinetic and thermo sources
uncertainty.extract_sources_from_model()
uncertainty.assign_parameter_uncertainties()

In [ ]:
# Create a giant dictionary with all of the reaction family information in it
auto_gen_families = {}
for family_name in uncertainty.database.kinetics.families.keys():
    if family_name == 'Intra_R_Add_Endocyclic' or family_name == 'Intra_R_Add_Exocyclic':
        continue
    if uncertainty.database.kinetics.families[family_name].auto_generated and family_name not in auto_gen_families.keys():
        auto_gen_families[family_name] = uncertainty.database.kinetics.families[family_name].rules.get_entries()
        auto_gen_families[f'{family_name}_labels'] = [entry.label for entry in uncertainty.database.kinetics.families[family_name].rules.get_entries()]
        auto_gen_families[f'{family_name}_rxn_map'] = uncertainty.database.kinetics.families[family_name].get_reaction_matches(
            thermo_database=uncertainty.database.thermo,
            remove_degeneracy=True,
            get_reverse=True,
            exact_matches_only=False,
            fix_labels=True)

## Just Look at Disproportionation

In [ ]:
# Make a set of all the training reactions in the Disproportionation Family Tree
family_name = 'Disproportionation'
training_reactions = set()
for node_name in uncertainty.database.kinetics.families[family_name].rules.entries.keys():
    for training_rxn in auto_gen_families[f'{family_name}_rxn_map'][node_name]:
        training_reactions.add(training_rxn)
training_reactions = list(training_reactions)
print(len(training_reactions), 'training reactions')


In [ ]:
for training_reaction in training_reactions:
    print(training_reaction)

In [ ]:
training_reaction_scores = np.zeros(len(training_reactions))
for i, reaction in enumerate(uncertainty.reaction_list):
#     reaction_sensitivity = 1.0
    reaction_sensitivity = sensitivities[i]

    try:
        if reaction.family == family_name:
            # Get the node
            node_name = ''
#                 print(i, reaction, node_name)
            print(i, reaction)
            if 'Rate Rules' in uncertainty.reaction_sources_dict[uncertainty.reaction_list[i]].keys():

                node_name = uncertainty.reaction_sources_dict[uncertainty.reaction_list[i]]['Rate Rules'][1]['node']
                node_sensitivities = unpack_sensitivity(uncertainty.database.kinetics.families['Disproportionation'].rules.entries[node_name][0].long_desc)
                    
                # find each training reaction in the node
                for t, train in enumerate(auto_gen_families[f'{family_name}_rxn_map'][node_name]):
                    # Get the partial sensitivity dNode / dtraining_reaction (A factor)
                    # equal division
                    partial_sensitivity = 1.0 / len(auto_gen_families[f'{family_name}_rxn_map'][node_name])
                    
                    # use calculated derivatives
#                     partial_sensitivity = node_sensitivities[t]['dA']
                    # use the chain rule and multiply by this mechanism reaction's sensitivity
                    total_sensitivity = reaction_sensitivity * partial_sensitivity
                    
                    for j, training_rxn in enumerate(training_reactions):
                        if training_rxn.is_isomorphic(train):
                            training_reaction_scores[j] += total_sensitivity  # or whatever the reaction sensitivity is
                            print('\t', training_rxn, '\t', np.round(total_sensitivity, 8))
                            break
                    else:
                        raise ValueError('could not match training reaction in node')
                

            elif 'Training' in uncertainty.reaction_sources_dict[uncertainty.reaction_list[i]].keys():
                for j, training_rxn in enumerate(training_reactions):
                    if training_rxn.is_isomorphic(uncertainty.reaction_sources_dict[uncertainty.reaction_list[i]]['Training'][1].item):
                        training_reaction_scores[j] += reaction_sensitivity  # or whatever the reaction sensitivity is
                        print('matched training reaction exactly!')
                        break
            print()
            
    except AttributeError:
        continue


In [ ]:
# print the results
order = np.arange(0, len(training_reaction_scores))
train_rxn_order = [x for _, x in sorted(zip(training_reaction_scores, order))][::-1]


print('\tTraining Reaction\t\tSensitivity')
for i in range(0, len(training_reaction_scores)):
    index = train_rxn_order[i]
    print(index,'\t', training_reactions[index], '\t', training_reaction_scores[index])

